EDA

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 1. โหลดไฟล์
df = pd.read_csv("co2_data_2000_2024_clear_dev_level.csv")

# 2. เคลียร์ค่า missing ที่จำเป็น
df_clean = df.dropna(subset=["gdp_per_capita", "dev_level"])
df_clean["year"] = df_clean["year"].astype(int)

# 3. ดู dev_level ที่มีในดาต้า (ช่วยให้รู้ว่ามีกี่กลุ่ม ชื่ออะไรบ้าง)
print("dev_level ที่มีในดาต้า:", df_clean["dev_level"].unique())

# 4. เลือก dev_level ที่อยากพล็อตเองตรงนี้ 👇
#    - ถ้าอยากเอาทุกกลุ่ม ให้ตั้ง dev_levels_to_plot = [] หรือ None
dev_levels_to_plot = ["Developed"]   # <-- แก้ชื่อกลุ่มตรงนี้

if dev_levels_to_plot:  # ถ้ามีระบุรายชื่อ จะ filter ตามนั้น
    df_filtered = df_clean[df_clean["dev_level"].isin(dev_levels_to_plot)]
else:                   # ถ้าไม่ระบุ ใช้ทุก dev_level
    df_filtered = df_clean.copy()

# กันเคสเลือกแล้วไม่มีข้อมูล
if df_filtered.empty:
    raise ValueError("ไม่มีข้อมูลสำหรับ dev_level ที่เลือก ลองเช็คชื่อใน print ข้างบนอีกทีนะ")

# 5. คำนวณค่าเฉลี่ย GDP per capita ต่อปี-ต่อ dev_level
df_group = (
    df_filtered
    .groupby(["year", "dev_level"])["gdp_per_capita"]
    .mean()
    .reset_index()
    .sort_values("year")
)

# 6. วาดกราฟ line plot แยกเส้นตาม dev_level ที่เลือก
plt.figure(figsize=(12, 6))

for level, sub in df_group.groupby("dev_level"):
    plt.plot(
        sub["year"],
        sub["gdp_per_capita"],
        marker="o",
        linewidth=2,
        label=str(level)
    )

plt.title("Average GDP per capita per year by selected development levels")
plt.xlabel("Year")
plt.ylabel("Average GDP per capita (USD)")
plt.grid(True, linestyle="--", alpha=0.4)
plt.legend(title="Development level")
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 1. โหลดไฟล์
df = pd.read_csv("co2_data_2000_2024_clear_dev_level.csv")

# ------------------------------
# 🧩 CONFIG ส่วนที่เราแก้เองได้
# ------------------------------

# 1) เลือกตัวแปรบนแกน Y ที่จะ plot
# เช่น "gdp_per_capita", "co2_per_capita", "co2", "population" ฯลฯ
value_col = "oil_co2"   # <--- แก้ตรงนี้เวลาอยากเปลี่ยนตัวแปร

# 2) เลือก dev_level ที่อยาก plot
# ถ้าอยากเอาทุกกลุ่มให้ตั้ง dev_levels_to_plot = [] หรือ None
#"No-Develop", "Developing", "Developed"
dev_levels_to_plot = ["Developed"]   # <--- แก้เป็นกลุ่มที่มีในดาต้า

# 3) เลือกช่วงปีที่อยากดู
min_year = 2000
max_year = 2022             # <--- ตรงนี้เลย เอาถึงปี 2022

# ------------------------------
# ตรวจสอบคอลัมน์ & เตรียมข้อมูล
# ------------------------------
if value_col not in df.columns:
    raise ValueError(f"คอลัมน์ '{value_col}' ไม่มีใน DataFrame\nมีคอลัมน์: {list(df.columns)}")

# ลบ missing ของคอลัมน์ที่จำเป็น
df_clean = df.dropna(subset=[value_col, "dev_level", "year"])

# แปลงปีให้เป็น int เผื่อเป็น float/str
df_clean["year"] = df_clean["year"].astype(int)

# ✅ กรองช่วงปีตามที่กำหนด
df_clean = df_clean[(df_clean["year"] >= min_year) & (df_clean["year"] <= max_year)]

# ดู dev_level ทั้งหมดไว้เช็คชื่อ
print("dev_level ที่มีในดาต้า:", df_clean["dev_level"].unique())

# filter ตาม dev_level ถ้ากำหนด
if dev_levels_to_plot:
    df_filtered = df_clean[df_clean["dev_level"].isin(dev_levels_to_plot)]
else:
    df_filtered = df_clean.copy()

if df_filtered.empty:
    raise ValueError("ไม่มีข้อมูลสำหรับ dev_level หรือช่วงปีที่เลือก ลองเช็คอีกทีนะ")

# ------------------------------
# groupby ปี + dev_level แล้วเอาค่าเฉลี่ยของ value_col
# ------------------------------
df_group = (
    df_filtered
    .groupby(["year", "dev_level"])[value_col]
    .mean()
    .reset_index()
    .sort_values("year")
)

# ------------------------------
# วาดกราฟ
# ------------------------------
plt.figure(figsize=(12, 6))

for level, sub in df_group.groupby("dev_level"):
    plt.plot(
        sub["year"],
        sub[value_col],
        marker="o",
        linewidth=2,
        label=str(level)
    )

plt.title(f"Average {value_col} per year by selected development levels\n({min_year}-{max_year})")
plt.xlabel("Year")
plt.ylabel(value_col)
plt.grid(True, linestyle="--", alpha=0.4)
plt.legend(title="Development level")
plt.tight_layout()
plt.show()
